In [4]:
import os

In [5]:
# Specify the directory containing the YOLOv5 labels
animal_labels_dir = '/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Projects/CameraTraps/Data/Goat/test/'

# Initialize an empty species mapping dictionary
species_mapping = {}

In [6]:
# Function to process the YOLO label file and update the class IDs
def process_label_file(label_file):
    with open(label_file, 'r') as f:
        lines = f.readlines()

    modified_lines = []

    # Extract the species name from the label file path
    species_name = label_file.split('/')[-3]  # Extract the second from the last folder

    # Check if the species name already exists in the mapping dictionary
    if species_name not in species_mapping:
        # Generate a new class ID for the species and add it to the mapping dictionary
        new_class_id = len(species_mapping)
        species_mapping[species_name] = new_class_id
    else:
        # Retrieve the existing class ID for the species from the mapping dictionary
        new_class_id = species_mapping[species_name]

    # Loop through each line in the label file
    for line in lines:
        class_id, x_center, y_center, width, height = map(float, line.split())

        # Modify the line with the updated class ID
        modified_line = f"{int(new_class_id)} {x_center} {y_center} {width} {height}"
        modified_lines.append(modified_line)

    # Write the modified lines back to the label file
    with open(label_file, 'w') as f:
        f.write('\n'.join(modified_lines))


# Loop through each subdirectory in the "animal" labels directory
for root, dirs, files in os.walk(animal_labels_dir):
    for file in files:
        if file.endswith('.txt'):
            label_file = os.path.join(root, file)
            process_label_file(label_file)

# Save the species mapping to a .txt file
species_mapping_file = 'classes.txt'
with open(animal_labels_dir+species_mapping_file, 'w') as f:
    for species_name, class_id in species_mapping.items():
        f.write(f"{species_name}: {class_id}\n")